## Построение графов взаимодействий

**Задание 1.** Прочитайте из файла ```Грибоедов Александр. Горе от ума.txt``` пьесу. В начале текстового файла будут перечисленны героини пьесы, создайте список основных действующих лиц. Далее разделите пьесу на действия, а действия на явления. Для каждого явления получите список участвующих в нём персонажей и длину его реплики в словах. Какую структуру вы выберите для хранения данной информации?

**Задание 2**. После выполнения первого задания у вас должна быть реализована структура, из которой вы сможете для каждого действия и каждого явления определить, какие персонажи в нём участвовали. Выведите на экран список уникальных персонажей, которых вам удалось выделить в пьесе. Сколько таких персонажей? 

Замечание: Список уникальных персонажей вы должно получить с помощью одной строчки кода =) 

**Задание 3.** Формирование матрицы взаимодействий.

Будем считать, что персонажи вступили во взаимодействие, если они вместе участвовали в явлении. 

Пусть у нас $N$ уникальных персонажей. Создайте квадратную матрицу из нулей разъмера N. Каждому персонажу присвойте номер от $0$ до $N-1$.

Тогда, если в некотором явление участвовал список персонажей $\{n_1, n_2, n3\}$, то значения матрицы взаимодействий $A$ обновляются следующим образом:

$$A[i, j] = A[i, j] + 1$$ где $i \neq j \in \{n_1, n_2, n3\}$

Уникальные номера персонажам присвойте с помощью [LabelEncoder](sklearn.preprocessing.LabelEncoder)

Матрицу взаимосвязей визуализируйте с помощью [matshow](https://matplotlib.org/api/_as_gen/matplotlib.pyplot.matshow.html#matplotlib.pyplot.matshow). А также воспользуйтесь [colorbar](https://matplotlib.org/api/_as_gen/matplotlib.pyplot.colorbar.html) и [xticks/yticks](https://matplotlib.org/api/_as_gen/matplotlib.pyplot.xticks.html)

Какие персонажи чаще всего взаимодействуют?

**Задание 4.** Постройте граф взаимасвязей с помощью [NetworkX](https://pypi.org/project/networkx/) ([Хабрахабра](https://habrahabr.ru/post/125898/))

**Задание 5.** Вспомним, что в первом задании мы так же сохраняли информацию о том, сколько слов произнёс каждый персонаж. 

Пусть в некотором явлении участвовало некоторое подмножество персонажей и мы знаем их вес -- количество слов: $\{n_1:w_1, n_2:w_2, n_3:w3\}$. Также допустим, что $w_1 > w_2, w_1 > w3$ -- первый действующий персонаж произнёс больше всех слов. 

Тогда считаем, что персонаж $n_1$ воздействовал на персонажей $n_2$ и $n_3$. Преобразуем матрицу взаимодействий следующим образом:

$$ A[n_1, j] = A[n_1, j] + 1 $$ где $j \in \{n_2, n_3\}$

Таким образом матрица $A$ получится не симметричной, а соответсвующий граф взаимодействий направленным.

Нарисуте граф взаимодействий